In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import math

# ==========================================
# ЧАСТЬ 1: ДАННЫЕ (ОЦИФРОВАНЫ С ВАШИХ СКРИНШОТОВ)
# ==========================================

# Данные из Таблицы А.6 (Показатели деятельности 40 предприятий)
# Нам нужны только столбцы 1 и 13 для Варианта 4, но я добавлю структуру, чтобы было понятно.
# Значения переписаны со скриншотов 222, 223, 224.

data_task5_values = {
    # "Продуктивність праці" (Показатель 1)
    1: [
        9.26, 9.38, 12.11, 10.81, 9.35, 9.87, 8.17, 9.12, 5.88, 6.3,
        6.22, 5.49, 6.5, 6.61, 4.32, 7.37, 7.02, 8.25, 8.15, 8.72,
        7.65, 8.77, 7.0, 11.06, 9.02, 13.28, 9.27, 6.7, 6.69, 9.42,
        7.24, 5.39, 5.61, 5.59, 6.57, 6.54, 4.23, 5.22, 18.0, 11.03
    ],
    # "Середньорічний фонд зарплати" (Показатель 13)
    13: [
        47750, 50391, 43149, 41089, 14257, 22061, 52509, 14903, 11152, 16821,
        19459, 12973, 50907, 6920, 5736, 26705, 20068, 11487, 32029, 18946,
        19074, 18452, 12096, 7888, 7566, 94697, 29626, 11688, 21955, 12243,
        48141, 20122, 7612, 27404, 39648, 43799, 6235, 11524, 17309, 22225
    ]
}
df_task5 = pd.DataFrame(data_task5_values)

# Данные из Таблицы Б.4 (Оценки экспертов для 13 компетенций)
# Строки - компетенции, Столбцы - Эксперты 1-8.
data_task6_values = {
    'Comp': [
        'Орієнтація на результат', 'Організованість', 'Впливовість', 'Комунікабельність',
        'Впевненість у собі', 'Керування конфліктами', 'Адаптовність', 'Командна робота',
        'Орієнтація на клієнта', 'Активне слухання', 'Відкритість', 'Керування ризиками', 'Гнучкість'
    ],
    1: [5, 5, 5, 3, 5, 4, 4, 5, 5, 4, 5, 5, 4],
    2: [5, 5, 5, 5, 4, 5, 5, 3, 4, 4, 4, 4, 3],
    3: [5, 5, 5, 5, 4, 5, 4, 5, 3, 5, 4, 4, 5],
    4: [5, 5, 4, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4],
    5: [4, 5, 5, 2, 5, 4, 3, 5, 5, 3, 5, 4, 5],
    6: [5, 5, 4, 4, 5, 5, 5, 3, 4, 5, 4, 4, 3],
    7: [5, 5, 3, 5, 4, 5, 4, 5, 4, 2, 4, 5, 5],
    8: [5, 3, 4, 5, 4, 5, 4, 5, 5, 4, 4, 3, 4]
}
df_all_experts = pd.DataFrame(data_task6_values).set_index('Comp')


# ==========================================
# ЧАСТЬ 2: РЕАЛИЗАЦИЯ АЛГОРИТМОВ (ПО МЕТОДИЧКЕ)
# ==========================================

def get_ranks(data):
    """Присвоение рангов (средние для связок)"""
    return data.rank(method='average')

def calculate_ties_correction(ranks):
    """Поправка T на связанные ранги: T = 1/12 * sum(t^3 - t)"""
    values, counts = np.unique(ranks, return_counts=True)
    T = 0
    for t in counts:
        if t > 1:
            T += (t**3 - t) / 12
    return T

# --- ЗАДАНИЕ 5: СПИРМЕН ---
def spearman_correlation_manual(x, y, alpha=0.05):
    n = len(x)
    rx = get_ranks(x)
    ry = get_ranks(y)

    d_sq_sum = np.sum((rx - ry)**2)
    Tx = calculate_ties_correction(rx)
    Ty = calculate_ties_correction(ry)

    # Формула (2.2) для связанных рангов
    num = (1/6)*(n**3 - n) - d_sq_sum - Tx - Ty
    den_x = (1/6)*(n**3 - n) - 2*Tx
    den_y = (1/6)*(n**3 - n) - 2*Ty

    if den_x * den_y == 0:
        rc = 0
    else:
        rc = num / np.sqrt(den_x * den_y)

    # t-критерий Стьюдента
    if abs(rc) >= 1:
        t_stat = np.inf
    else:
        t_stat = abs(rc) * np.sqrt((n - 2) / (1 - rc**2))

    df = n - 2
    t_crit = stats.t.ppf(1 - alpha/2, df)

    return rc, t_stat, t_crit, t_stat > t_crit

# --- ЗАДАНИЕ 5: КЕНДАЛЛ ---
def kendall_correlation_manual(x, y, alpha=0.05):
    n = len(x)
    df_temp = pd.DataFrame({'x': x, 'y': y}).sort_values(by='x')
    y_sorted = df_temp['y'].values

    # S = P - Q
    S = 0
    for i in range(n):
        for j in range(i + 1, n):
            if y_sorted[j] > y_sorted[i]: S += 1
            elif y_sorted[j] < y_sorted[i]: S -= 1

    # Поправки (через V или T)
    rx = get_ranks(df_temp['x']).values
    ry = get_ranks(pd.Series(y_sorted)).values

    def calc_V(ranks):
        values, counts = np.unique(ranks, return_counts=True)
        V = 0
        for t in counts:
            if t > 1: V += (t * (t - 1)) / 2
        return V

    Vx = calc_V(rx)
    Vy = calc_V(ry)

    denom = np.sqrt((0.5*n*(n-1) - Vx) * (0.5*n*(n-1) - Vy))
    rk = S / denom if denom != 0 else 0

    # Z-тест (формула 2.7)
    z_stat = rk / np.sqrt((2 * (2*n + 5)) / (9 * n * (n - 1)))
    u_crit = stats.norm.ppf(1 - alpha/2)

    return rk, abs(z_stat), u_crit, abs(z_stat) > u_crit

# --- ЗАДАНИЕ 6: КОНКОРДАЦИЯ ---
def concordance_coefficient(expert_matrix, alpha=0.05):
    n = expert_matrix.shape[0] # объекты (13)
    m = expert_matrix.shape[1] # эксперты (3)

    ranked_matrix = expert_matrix.apply(get_ranks, axis=0)
    Ri = ranked_matrix.sum(axis=1)
    R_mean = Ri.mean()
    S = np.sum((Ri - R_mean)**2)

    sum_Ti = 0
    for col in ranked_matrix.columns:
        sum_Ti += calculate_ties_correction(ranked_matrix[col])

    numerator = S
    denominator = (1/12)*(m**2)*(n**3 - n) - m * sum_Ti
    W = numerator / denominator if denominator != 0 else 0

    # Хи-квадрат (формула 2.11)
    chi2_calc = m * (n - 1) * W
    df_chi = n - 1
    chi2_crit = stats.chi2.ppf(1 - alpha, df_chi)

    return W, chi2_calc, chi2_crit, chi2_calc > chi2_crit


# ==========================================
# ЧАСТЬ 3: РЕШЕНИЕ ДЛЯ ВАРИАНТА 4
# ==========================================

print("=== РЕЗУЛЬТАТЫ ДЛЯ ВАРИАНТА 4 ===\n")

# --- ЗАДАНИЕ 5 ---
print(">>> ЗАДАНИЕ 5: Корреляция признаков")
# Для варианта 4: Признаки 1 и 13 (см. Табл. Б.3)
X_var = df_task5[1]
Y_var = df_task5[13]

print(f"Анализ признаков: 1 (Продуктивність) и 13 (Фонд зарплати)")
print(f"Количество наблюдений n = {len(X_var)}")

# Спирмен
rc, t_calc, t_table, sig_s = spearman_correlation_manual(X_var, Y_var)
print(f"\n1. Коэффициент Спирмена (rc): {rc:.4f}")
print(f"   Проверка: t_расч = {t_calc:.4f} {' > ' if sig_s else ' <= '} t_крит = {t_table:.4f}")
print(f"   Вывод: {'Зв’язок статистично значущий' if sig_s else 'Зв’язок не значущий'}")

# Кендалл
rk, z_calc, u_table, sig_k = kendall_correlation_manual(X_var, Y_var)
print(f"\n2. Коэффициент Кендалла (rk): {rk:.4f}")
print(f"   Проверка: z_расч = {z_calc:.4f} {' > ' if sig_k else ' <= '} u_крит = {u_table:.4f}")
print(f"   Вывод: {'Зв’язок статистично значущий' if sig_k else 'Зв’язок не значущий'}")

# Вывод о линейности
print("\n3. Характер связи:")
if abs(rk) > abs(rc):
    print("   |rk| > |rc| -> Зависимость скорее нелинейная")
else:
    print("   |rc| >= |rk| -> Зависимость ближе к линейной")


# --- ЗАДАНИЕ 6 ---
print("\n" + "="*40 + "\n")
print(">>> ЗАДАНИЕ 6: Узгодженість експертів")
# Для варианта 4: Эксперты 1, 3, 7 (см. Табл. Б.5)
expert_cols = [1, 3, 7]
print(f"Выбранные эксперты: {expert_cols}")

df_experts_var4 = df_all_experts[expert_cols]

W_val, chi2_calc, chi2_table, sig_w = concordance_coefficient(df_experts_var4)

print(f"\nКоэффициент конкордации (W): {W_val:.4f}")
print(f"Проверка значимости (Хи-квадрат):")
print(f"   Chi2_расч = {chi2_calc:.4f}")
print(f"   Chi2_крит = {chi2_table:.4f}")
print(f"Вывод: {'Думки експертів узгоджені (W значущий)' if sig_w else 'Узгодженість думок низька (W не значущий)'}")

=== РЕЗУЛЬТАТЫ ДЛЯ ВАРИАНТА 4 ===

>>> ЗАДАНИЕ 5: Корреляция признаков
Анализ признаков: 1 (Продуктивність) и 13 (Фонд зарплати)
Количество наблюдений n = 40

1. Коэффициент Спирмена (rc): 0.3274
   Проверка: t_расч = 2.1359  >  t_крит = 2.0244
   Вывод: Зв’язок статистично значущий

2. Коэффициент Кендалла (rk): 0.2231
   Проверка: z_расч = 2.0273  >  u_крит = 1.9600
   Вывод: Зв’язок статистично значущий

3. Характер связи:
   |rc| >= |rk| -> Зависимость ближе к линейной


>>> ЗАДАНИЕ 6: Узгодженість експертів
Выбранные эксперты: [1, 3, 7]

Коэффициент конкордации (W): 0.3214
Проверка значимости (Хи-квадрат):
   Chi2_расч = 11.5704
   Chi2_крит = 21.0261
Вывод: Узгодженість думок низька (W не значущий)
